# Running Experiments

This notebook shows how to use the `scripts` to execute experiments.

For lighter-weight fiddling, the `critfinder` module provides a better API.
The figure generation notebook contains an example under the **Wall Time** section.

In [ ]:
import os
import sys

path_above = os.path.abspath("..")

%env PYTHONPATH={path_above}

sys.path.append(path_above)

In [ ]:
import critfinder.utils.util

## Setup

We begin by setting some path variables and hyperparameters.

`optimizations_path` is a path to a directory that will contain `optimizations`, which are combinations of an `optimizer` with a problem to run on. The problem is defined by its `data` and its `network`, both of which have configurations that are stored as `.json` files for easy reconstruction.

These experiments are done with the presumption that the data has already been generated, and is located at the `data_path` below.

In [ ]:
optimizations_path = "../results/optimizations"
data_path = "../data/subtract_zero_center_gauss_16_linspace/xs.npy"

num_optimizer_steps = 10000
num_gnm_steps = 50000
num_newton_steps = 500

## Optimization Experiment

Both experiments are executed in the same fashion: first, a `setup` python script is run to create all of the configuration files for each component of the experiment: the data, the network, the optimizer/finder, etc.

The `setup_XYZ_experiment.py` scripts take a very large number of keyword arguments, so they are equipped with documentation. Run `setup_XYZ_experiment.py -h` to see them.

### Setup

Optimizers have identification strings which are either random or provided by the user. This is used to name the directory in which, by default, all of the configuration files are stored.

In [ ]:
optimizer_ID = critfinder.utils.util.random_string(6)

print(optimizer_ID)

In [ ]:
optimization_path = os.path.join(optimizations_path, optimizer_ID)

In [ ]:
!python setup_optimization_experiment.py --results_path {optimizations_path} \
    --ID {optimizer_ID} \
    --data_path {data_path} \
    --layers 4 \
    --k 16 \
    --optimizer "gd"\
    --optimizer_lr 0.01\
    --zero_centering "subtract_mean"

Optimizers can be run multiple times from different initial choices of parameter value, with the resulting trajectory saved separately each time. In order to do so, we need to provide a different `trajectory_path` to the `run_optimizer_experiment.py` script each time it is executed. This is handled by the `optimizer_trajectory_increment` variable here, but one could also use random IDs.

For downstream scripts, the default, when there is one, is to look for a trajectory at `0000.npz` within a given folder.

In [ ]:
optimizer_trajectory_increment = 3

### Run

We now run the experiment by passing the appropriate paths and a step count to `run_optimization_experiment`. If given an `optimization_path`, it will look for the configuration `.json` files in that directory.

In [ ]:
trajectory_path = os.path.join(optimization_path, "trajectories",
                               str(optimizer_trajectory_increment).zfill(4) + ".npz")

!python run_optimization_experiment.py {num_optimizer_steps} \
    --optimization_path {optimization_path} --trajectory_path {trajectory_path}
    
optimizer_trajectory_increment += 1

## Critfinder Experiments

Critfinder experiments are executed in much the same fashion: `setup` and then `run`.

### Setup

The most important variable is the `finder_str`, which identifies the critfinding algorithm. Current choices are `gnm` (gradient norm minimization, as in Pennington and Bahri), `newtonMR` (`m`in`r`es, by Roosta et al.), and `newtonTR` (trust region, as in Dauphin et al.).

In [ ]:
finder_str = "newtonMR"

I've elected to organize around particular optimization problems, which combine loss surfaces and optimizers on those loss surfaces (which generate trajectories that have candidate initial thetas).

In [ ]:
critfinders_path = os.path.join(optimization_path, "critfinders")

Critfinders are setup using the `setup_critfinder.py` script.

The argument structure is quite different depending on which method is being called: for example, `gnm` needs `minimizer`, either `g`radient `d`escent, `momentum`, or `b`ack`t`racking `l`ine `s`earch, while `newtonXY` methods do not.

In [ ]:
def setup_critfinder(critfinder_ID, finder_str, optimizer_trajectory_increment, theta_pertub=None):

    trajectory_file = str(optimizer_trajectory_increment).zfill(4) + ".npz"

     if noise_level is not None:
        if finder_str == "gnm":
            !python setup_critfinder_experiment.py {optimization_path} {finder_str} \
                --base_critfinders_path {critfinders_path} \
                --ID {critfinder_ID} \
                --minimizer "btls" \
                --init_theta "uniform_f" \
                --theta_perturb {theta_perturb} \
                --trajectory_file {trajectory_file}
            
        if "newton" in finder_str:
            !python setup_critfinder_experiment.py {optimization_path} {finder_str} \
                --base_critfinders_path {critfinders_path} \
                --ID {critfinder_ID} \
                --init_theta "uniform_f" \
                --theta_perturb {theta_perturb} \
                --trajectory_file {trajectory_file} \
                --gamma_mx 2 \
                --gamma_k 10
    else:
        
        if finder_str == "gnm":
            !python setup_critfinder_experiment.py {optimization_path} {finder_str} \
                --base_critfinders_path {critfinders_path} \
                --ID {critfinder_ID} \
                --minimizer "btls" \
                --init_theta "uniform_f" \
                --trajectory_file {trajectory_file}

        if "newton" in finder_str:
            !python setup_critfinder_experiment.py {optimization_path} {finder_str} \
                --base_critfinders_path {critfinders_path} \
                --ID {critfinder_ID} \
                --init_theta "uniform_f" \
                --trajectory_file {trajectory_file} \
                --gamma_mx 2 \
                --gamma_k 10

### Run

In [ ]:
runs_per_critfinder = 100

In [ ]:
starting_trajectory_increment = 0
ending_trajectory_increment = 1
    

for optimizer_trajectory_increment in range(starting_trajectory_increment, ending_trajectory_increment):

    critfinder_ID = finder_str + "_" + critfinder.utils.util.random_string(6)

    print(optimizer_trajectory_increment, critfinder_ID)

    setup_critfinder(critfinder_ID, finder_str, optimizer_trajectory_increment)

    for ii in range(0, runs_per_critfinder):
        print("\t" + str(ii))
        critfinder_path = os.path.join(critfinders_path, critfinder_ID)

        output_folder = os.path.join(critfinder_path, "output")

        output_path = os.path.join(output_folder, str(ii).zfill(4) + ".npz")

        if finder_str == "gnm":
            !python run_critfinder_experiment.py {critfinder_path} {output_path} {num_gnm_steps}
        elif "newton" in finder_str:
            !python run_critfinder_experiment.py {critfinder_path} {output_path} {num_newton_steps}